# Нейронные сети

В этом уроке мы познакомимся с основами глубинного обучения на примере построения различных моделей для классификации изображений одежды по 10 различным классам.

Сначала импортируем нужные нам библиотеки

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.datasets import fashion_mnist

%matplotlib inline

Теперь загрузим данные. Заботливые ребята из гугла заранее любезно разделили датасет на `train` и `test` и обернули его в функцию библиотеки `tensorflow`, поэтому нам осталось лишь написать 1 строчку

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

Всегда полезно понимать, с какими данными мы будем работать, так что давайте взглянем на них поближе.

Размер обучающего датасета:

In [ ]:
train_images.shape

Размер тестового датасета:

In [ ]:
test_images.shape

Посмотрим, какие у нас есть метки классов

In [ ]:
np.unique(train_labels)

Теперь давайте взглянем на несколько изображений

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

n_rows = 4
n_cols = 10

fig, ax = plt.subplots(n_rows, n_cols, figsize=(14, 6))
for i in range(n_rows):
    for j in range(n_cols):
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])
        ax[i][j].imshow(train_images[i * n_rows + j], cmap=plt.cm.binary)
        ax[i][j].set_xlabel(class_names[train_labels[i * n_rows + j]])

Давайте, наконец, что-нибудь пообучаем. Для начала посмотрим, как себя покажет обычный линейный классификатор. 

Простейшие модели в `keras` реализуются с помощью класса `Sequential`. Класс есть просто обертка над списком слоев, через которые последовательно прокидывается инпут.

Сначала необходимо преобразовать двумерное изображение в одномерный массив с помощью слоя `Flatten`.

Затем добавим линейный слой с функцией активации `Softmax`.

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs = 20
batch_size = 500
model.fit(train_images, train_labels, 
          epochs=epochs, batch_size=batch_size,
          validation_data=(test_images, test_labels))

Довольно неплохой результат для классификации на 10 классов (`accuracy` рандомного классификатора была бы всего 10%). Но можно сделать лучше.

![layers](https://pbs.twimg.com/media/CYggEo-VAAACg_n.png:small)

Теперь время применять магию дип лернинга! Напишите модель, в которой будет хотя бы 3 линейных слоя. Во внутренних слоях используйте функцию активацию `ReLU` (`tf.nn.relu`). Выходной слой должен иметь 10 нейронов и функцию активации `softmax` (`tf.nn.softmax`). Будем считать, что модель должна набрать хотя бы 87% на тестовой выборке

In [ ]:
model = keras.Sequential([
    <YOUR CODE HERE>
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs = 20
batch_size = 500
model.fit(train_images, train_labels, 
          epochs=epochs, batch_size=batch_size,
          validation_data=(test_images, test_labels))

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                  100 * np.max(predictions_array),
                                  class_names[true_label]),
                                  color=color)

    
def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1]) 
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
predictions = model.predict(test_images)

num_rows = 5
num_cols = 4
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions, test_labels, test_images)
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
    plot_value_array(i, predictions, test_labels)
plt.show()

Теперь давайте обучим сверточную нейросеть. Для этого нам понадобится 2 новых слоя: `keras.layers.Conv2D` -- сам сверточный слой, и `keras.layers.MaxPool2D` -- слой пулинга. Здесь код уже написан за вас, просто запускайте все клетки последовательно.

In [ ]:
model = keras.Sequential([
    keras.layers.Reshape((28, 28, 1)),
    
    keras.layers.Conv2D(32, kernel_size=3, padding='same', activation=tf.nn.relu),
    keras.layers.Dropout(rate=0.3),
    keras.layers.MaxPool2D(2),
    
    keras.layers.Conv2D(64, kernel_size=3, padding='same', activation=tf.nn.relu),
    keras.layers.Dropout(rate=0.3),
    keras.layers.MaxPool2D(2),
    
    keras.layers.Conv2D(128, kernel_size=7, padding='same', activation=tf.nn.relu),
    keras.layers.Dropout(rate=0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs = 20
batch_size = 500
model.fit(train_images, train_labels, 
          epochs=epochs, batch_size=batch_size,
          validation_data=(test_images, test_labels))

In [ ]:
predictions = model.predict(test_images)

num_rows = 5
num_cols = 4
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions, test_labels, test_images)
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
    plot_value_array(i, predictions, test_labels)
plt.show()